# Region-scale glacier analysis


The previous notebook demonstrated using xarray to analyze surface velocity data for an individual glacier. This notebook will show how we can examine spatial variability in surface velocity within a group of glaciers. To do this we will use xarray as well as **geopandas**, **geocube**, and **pandas**. We will start by using `make_geocube()` to rasterize a vector object in the shape of an **ITS_LIVE** velocity raster object. We will then use the rasterized vector to group the **ITS_LIVE** object by individual glaciers and then calculate summary statistics of surface velocity for each glacier. The goal in this work flow is to end up with a **pandas dataframe** where each row is an individual glacier and columns for various surface velocity summary statistics. 

*Learning goals*
- rasterizing vector data
- organizing and re-arranging data with xarray
- `groupby()` for zonal statistics
- converting from xarray to pandas

In [ ]:
import os
import json
import urllib.request
import numpy as np
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
import pandas as pd
import seaborn as sns 

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

from shapely.geometry import Polygon
from shapely.geometry import Point
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy
import cartopy.feature as cfeature

from geocube.api.core import make_geocube

%config InlineBackend.figure_format='retina'

In [ ]:
import itslivetools

## Accessing ITS_LIVE data

In [ ]:
with urllib.request.urlopen('https://its-live-data.s3.amazonaws.com/datacubes/catalog_v02.json') as url_catalog:
    itslive_catalog = json.loads(url_catalog.read().decode())
itslive_catalog.keys()

In [ ]:
url = itslivetools.find_granule_by_point(itslive_catalog, [84.56, 28.54])
url

In [ ]:
dc = itslivetools.read_in_s3(url[0])
dc

The `mid_date` dimension of the `dc` object isn't in chronlogical order, so let's sort by this dimension:

In [ ]:
dc = dc.sortby('mid_date')

In [ ]:
dc

## Vector data 

In [ ]:
se_asia = gpd.read_file('https://github.com/scottyhq/rgi/raw/main/15_rgi60_SouthAsiaEast.gpkg')
se_asia.head(3)

How many glaciers are in this dataframe?

In [ ]:
se_asia['RGIId'].nunique()

What coordinate reference system is this dataframe in? 

In [ ]:
se_asia.crs

The vector dataset is in WGS 84, meaning that its coordinates are in degrees latitude and longitude rather than meters N and E. We will project this dataset to match the projection of the netcdf dataset.

## Handling projections

Let's project this dataframe to match the CRS of the itslive dataset

In [ ]:
#project rgi data to match itslive
se_asia_prj = se_asia.to_crs('EPSG:32645') #we know the epsg from looking at the 'spatial epsg' attr of the mapping var of the dc object
se_asia_prj.head(3)

Give each glacier (row) a unique integer key that is related to that glacier's RGIId. We will use this later. Be careful that the `RGI_int` column is composed of **integers** not strings.

In [ ]:
se_asia_prj['RGI_int'] = se_asia_prj['RGIId'].str.slice(9,).replace('.','_')
se_asia_prj['RGI_int'] = se_asia_prj.RGI_int.apply(lambda x: int('15' + x))
se_asia_prj.RGI_int.dtype

To start with, we will look only at glaciers larger in area than 5km2. Subset the dataset to select for those glaciers

In [ ]:
se_asia_prj = se_asia_prj.loc[se_asia_prj['Area'] > 5.]
se_asia_prj.head()

Next, want to subset the RGI dataset by the spatial extent of the ITS_LIVE data.
First, get the bbox of the ITS_LIVE data as a vector

In [ ]:
dc_bbox = itslivetools.get_bbox_single(dc)

Project it to local UTM to match the RGI geodataframe and extract the coordinate values from the geometry column

In [ ]:
dc_bbox_prj = dc_bbox.to_crs('EPSG:32645')


Subset RGI dataset:
To do this we will use a [spatial join](https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin.html). Here we use an `inner join` but there are various methods to customize the spatial join operation. Find more info [here](https://geopandas.org/en/stable/gallery/spatial_joins.html). 

In [ ]:
rgi_sub = gpd.sjoin(se_asia_prj, dc_bbox_prj, how='inner')
# need to set the type as string here bc for some reason its object intead of str
rgi_sub["RGIId"] = rgi_sub.RGIId.astype("string")
rgi_sub.head()

In [ ]:
rgi_sub['RGIId'].values

Need to write crs of dc object?

In [ ]:
dc = dc.rio.write_crs(f"epsg:{dc.mapping.attrs['spatial_epsg']}", inplace=True)

In [ ]:
rgi_sub = rgi_sub.drop('index_right', axis=1)

In [ ]:
rgi_sub.head()

Now, use the `make_geocube()` function. This essentially takes a vector object (`rgi_sub`) and rasterizes it, returning an xarray object with the same structure as the object you provide for the `like =` argument (in our case that is `dc`).

In [ ]:
out_grid = make_geocube(
    vector_data = rgi_sub,
    measurements = ["RGI_int"],
    like = dc
)
out_grid

Now each glacier in the geodataframe `rgi_sub` has been coded with a unique integer value that corresponds to that glacier's Randolph Glacier Inventory ID. 

In [ ]:
out_grid.RGI_int.plot()

Next, merge the rasterized vector and the dataset containing the velocity data into an xarray dataset:

In [ ]:
out_grid['v'] = dc.v 
out_grid

Since we are mostly interested in examining spatial variability, let's take a temporal subset of the dataset to make the computation faster: 

In [ ]:
out_grid_sub = out_grid.sel(mid_date = slice('2015-01-01','2015-02-01')).compute()
out_grid_sub

In [ ]:
grouped_ID = out_grid_sub.drop('spatial_ref').groupby(out_grid_sub['RGI_int'])


In [ ]:
#compute zonal stats groupedd by ID
grid_mean_sp = grouped_ID.mean(dim=[...]).rename({'v': 'speed_mean'})
grid_min_sp = grouped_ID.min(dim=[...]).rename({'v': 'speed_min'})
grid_max_sp = grouped_ID.max(dim=[...]).rename({'v': 'speed_max'})
#grid_std_sp = grouped_ID.std(dim=['mid_date','stacked_y_x']).rename({'v': 'speed_std'}).compute()
    
#merge each zonal stat xr obj into a single xr obj, convert to pandas df
#zonal_stats = xr.merge([grid_mean_sp, grid_min_sp, grid_max_sp, grid_std_sp]).to_dataframe()
#zonal_stats = zonal_stats.reset_index()
#zonal_stats

Check if the data arrays are equal (the RGI_ints of each should be)

In [ ]:
grid_mean_sp.RGI_int.equals([grid_max_sp.RGI_int, grid_min_sp.RGI_int, grid_std_sp.RGI_int])


Looks like the issue is with `grid_std_sp`

In [ ]:
grid_mean_sp.RGI_int.equals(grid_std_sp.RGI_int)

Try to find the differences, ** stuck on this part.... 

or.... could just not use std

In [ ]:
#merge each zonal stat xr obj into a single xr obj, convert to pandas df
zonal_stats = xr.merge([grid_mean_sp, grid_min_sp, grid_max_sp]).to_dataframe()
zonal_stats = zonal_stats.reset_index()
zonal_stats = zonal_stats.drop(['mapping','spatial_ref'], axis=1)
zonal_stats

In [ ]:
rgi_itslive = rgi_sub.loc[rgi_sub['Area'] > 5.].merge(zonal_stats, on='RGI_int')


In [ ]:
rgi_itslive.columns

In [ ]:
len(set(rgi_itslive['RGIId']))

In [ ]:
fig, ax = plt.subplots()
rgi_itslive.plot.scatter(x='Aspect',y = 'speed_mean', c = 'darkblue', ax=ax)


In [ ]:
rgi_itslive.plot(column='speed_mean', legend=True)

In [ ]:
rgi_itslive.explore()